In [1]:
import numpy as np
import pandas as pd

In [159]:
def mrce(X, Y, lambda_1, lambda_2, epsilon):
    # initialize B and Gamma (matrix cov)
    B_t = np.zeros([X.shape[1], Y.shape[1]])
    Gamma_t = graphic_lasso(X, B_t, Y, lambda_1)
    Beta_ridge = np.abs(np.linalg.inv(X.T@X+lambda_2*np.diag(np.repeat(1,X.shape[1])))@(X.T@Y))
    B_t_new = coordinate_descent(X, Y, B_t, Gamma_t, lambda_2, epsilon)
    n = X.shape[1]
    k=0
    while np.sum(np.abs(B_t_new-B_t))>epsilon*np.sum(Beta_ridge):
        k+=1
        # print(k)
        B_t_new = coordinate_descent(X, Y, B_t, Gamma_t, lambda_2)
        Gamma_t = graphic_lasso(X, B_t_new, Y, lambda_1)
        #print(np.sum(np.abs(B_t_new-B_t)))
        #print(epsilon*np.sum(Beta_ridge))
    return B_t_new

In [160]:
def eliminar_fila_columna(X, i):
    n = X.shape[1]
    ind = np.arange(n)
    ind_new = ind[ind!=i]
    
    W = X[ind_new, :][:,ind_new]
    w = X[i,ind_new]
    w2 = X[i,i]
    return W, w, w2


def soft_threshold_operator(x, t, lim_sup=0):
    return np.sign(x)*np.max([lim_sup,np.abs(x)-t])

In [161]:
def coordinate_descent(X, Y, B_t, Gamma_t, lambda_2, epsilon=1e-5):
    n, p = X.shape
    Beta_ridge = np.abs(np.linalg.inv(X.T@X+lambda_2*np.diag(np.repeat(1,X.shape[1])))@(X.T@Y))
    
    S = X.T@X
    H = X.T@Y@Gamma_t
    #print(Gamma_t)
    B_t_new = np.copy(B_t)
    for r in range(B_t.shape[0]):
        for c in range(B_t.shape[1]):
            mu = 0
            for j in range(B_t.shape[0]):
                for k in range(B_t.shape[1]):
                    mu += B_t[j,k]*S[r,j]*Gamma_t[k,c]
            B_t_new[r,c] = soft_threshold_operator(B_t[r,c]+(H[r,c]-mu)/(S[r,r]*Gamma_t[c,c]), 
                                                   n*lambda_2/(S[r,r]*Gamma_t[c,c]),0)

    while np.sum(np.abs(B_t_new-B_t))>epsilon*np.sum(Beta_ridge):
        B_t = B_t_new
        for r in range(B_t.shape[0]):
            for c in range(B_t.shape[1]):
                mu = 0
                for j in range(B_t.shape[0]):
                    for k in range(B_t.shape[1]):
                        mu += B_t[j,k]*S[r,j]*Gamma_t[k,c]
                B_t_new[r,c] = B_t_new[r,c] = soft_threshold_operator(B_t[r,c]+(H[r,c]-mu)/(S[r,r]*Gamma_t[c,c]), 
                                                   n*lambda_2/(S[r,r]*Gamma_t[c,c]))
    return B_t_new

def graphic_lasso(X, B, Y, lambda_2, tol_sigma=1e-3, tol_beta=1e-3, max_iter=100):
    n, p = (Y-X@B).shape
    S = (Y-X@B).T@(Y-X@B)/n
    #S_datos = X.T@X
    Sigma = np.linalg.inv(S)
    Sigma_pre = np.zeros(Sigma.shape)
    Sigma_inv = np.copy(Sigma)
    count = 0
    while np.linalg.norm(Sigma - Sigma_pre) <= tol_sigma:
        Sigma_pre = np.copy(Sigma)
        for i in range(p):
            W, w, w2  = eliminar_fila_columna(Sigma_inv, i)
            _, s, s_ii = eliminar_fila_columna(S, i)
            
            beta = np.linalg.inv(W)@w
            beta_prev = np.zeros((p-1,1))
            while np.linalg.norm(beta - beta_prev) <= tol_beta:
                beta_prev = np.copy(beta)
                for j in range(p-1):
                    W_k = W[j, np.arange(p -1) != j][:,np.newaxis]
                    beta_k = beta[np.arange(p -1)!= j]
                    beta[j] = soft_threshold_operator(s[j]-W_k.T@beta_k, lambda_2)/W[j,j]
                    #print(np.linalg.norm(beta - beta_prev))
            ind = np.arange(p)
            ind_new = ind[ind!=i]
            #print(beta)
            w = W@beta
            w2 = s_ii+lambda_2
            Sigma_inv[i,ind_new] = w.T
            Sigma_inv[ind_new, i] = w.T
            Sigma_inv[i,i] = w2
            

            theta22 = 1.0 /(w2-w.T@beta)
            theta12 = -beta*theta22
            # print(theta12)
            
            
        count += 1
        if count>= max_iter:
            print("Maximo iteraciones")
            break
    Sigma = np.linalg.inv(Sigma_inv)

    return Sigma
    

In [162]:
from sklearn.datasets import make_regression
X,Y=make_regression(n_samples=100,n_features=20, n_targets=5,effective_rank=2,random_state=10)
#X_estandarizado=estandarizar.fit_transform(X)
#X=X_estandarizado
B_t = np.zeros([X.shape[1], Y.shape[1]])

mrce(X, Y, .002, 0.01, 2)

array([[ 8.44603176e+01, -9.13548495e+01, -5.77989411e+01,
        -6.97990239e+01,  4.37261727e+01],
       [ 1.82504973e+02, -5.23032331e+01, -2.59590314e+01,
         8.47965882e+00,  8.29518766e+00],
       [-2.04015770e+01,  4.72709965e+01,  2.09042842e+00,
         5.96384613e+01, -8.93615966e+00],
       [ 4.12438311e+01,  9.79200405e+00, -2.13221513e+01,
         3.49116507e+01,  1.21356625e+01],
       [-1.88188593e+02,  7.89651739e+01,  3.90837490e+01,
         1.13991240e+02, -2.59325842e+01],
       [-3.09424623e+02,  1.13890833e+02,  7.17843351e+01,
         2.34405567e+02, -6.81499051e+01],
       [ 7.41044127e+00, -4.77909110e+00,  1.38032979e+01,
        -6.92552033e+00, -9.63334297e+00],
       [ 3.76617973e+01, -3.26997103e+01, -5.91899584e+01,
         2.35597139e+01,  3.17177369e+01],
       [ 7.61894188e+01, -1.92650378e+01,  1.26451658e+00,
        -4.25202699e+01,  1.11598590e+01],
       [ 4.86206451e+00, -1.52242335e+01,  1.32160727e+01,
        -2.83703919e+01

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import GraphicalLasso


In [82]:
X_new = (Y-X@B_t)

In [125]:
true_cov = np.array([[0.8, 0.0, 0.2, 0.0],
                      [0.0, 0.4, 0.0, 0.0],
                      [0.2, 0.0, 0.3, 0.1],
                      [0.0, 0.0, 0.1, 0.7]])
np.random.seed(0)
X = np.random.multivariate_normal(mean=[0, 0, 0, 0],
                                   cov=true_cov,
                                   size=200) 
cov = GraphicalLasso(alpha=0.01,mode="cd").fit(X)

In [126]:
np.around(cov.covariance_, decimals=3)

array([[0.816, 0.049, 0.218, 0.019],
       [0.049, 0.364, 0.017, 0.034],
       [0.218, 0.017, 0.322, 0.093],
       [0.019, 0.034, 0.093, 0.69 ]])

In [59]:
X.shape

(200, 4)

In [56]:
B_t.shape

(20, 5)

In [57]:
Y.shape

(100, 5)

In [127]:
sigma = graphic_lasso(-X, np.diag(np.repeat(1,4)), np.zeros([200,4]), 0.01)
print(np.around(sigma, decimals=3))

[[[ 0.821  0.062  0.231  0.02 ]
  [ 0.062  0.366  0.027  0.05 ]
  [ 0.231  0.027  0.324  0.108]
  [ 0.02   0.05   0.108  0.711]]

 [[ 1.55  -0.198 -1.138  0.144]
  [-0.198  2.795 -0.029 -0.186]
  [-1.138 -0.029  4.102 -0.59 ]
  [ 0.144 -0.186 -0.59   1.505]]]


In [128]:
sigma = graphic_lasso2(X, 0.01)
print(np.around(sigma, decimals=1))

[[[ 1.5 -0.2 -1.   0.1]
  [-0.2  2.7 -0.  -0.1]
  [-1.  -0.   3.8 -0.5]
  [ 0.1 -0.1 -0.5  1.5]]

 [[ 0.8  0.1  0.2  0. ]
  [ 0.1  0.4  0.   0. ]
  [ 0.2  0.   0.3  0.1]
  [ 0.   0.   0.1  0.7]]]
